# Nginx

Nginx is a versatile tool that can be utilized as a web server, reverse proxy, load balancer, and much more. This page provides an overview of the fundamentals of working with Nginx.

**Sources:**

- [Nginx handbook](https://www.freecodecamp.org/news/the-nginx-handbook/).
- [Default variables available in NGINX](https://nginx.org/en/docs/varindex.html).

In [1]:
import requests
from IPython.display import HTML

For examples, we will run nginx in a docker container - the following cell runs nginx in a docker container.

In [2]:
!docker run -itd --name experiment_nginx --rm -p 80:80 nginx

docker: Error response from daemon: Conflict. The container name "/experiment_nginx" is already in use by container "f6b8d5c8cbdf081d2186986e41515347b163334ffe2b01955cb1034cb5f8978a". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


## Basics of configuration

Once the container is running, we can access nginx at `localhost:80`. The following cell shows python code that can display a basic nginx welcome page.

In [3]:
html_content = (
    requests
    .get("http://localhost:80")
    .content.decode("utf-8")
)
iframe = f"""
<iframe srcdoc='{html_content}' width='100%' height='600px' style='border:none;'></iframe>
"""
display(HTML(iframe))

Nginx is mainly controlled by its configuration files, so it's important to understand where they are. They are usually located in the `/etc/nginx/` folder. The following cell shows the contents of this folder.

In [4]:
!docker exec experiment_nginx ls -l /etc/nginx/

total 32
drwxr-xr-x 1 root root 4096 Jul 10 10:18 conf.d
-rw-r--r-- 1 root root 1007 May 28 13:22 fastcgi_params
-rw-r--r-- 1 root root 5349 May 28 13:22 mime.types
lrwxrwxrwx 1 root root   22 May 29 16:45 modules -> /usr/lib/nginx/modules
-rw-r--r-- 1 root root  648 May 29 16:45 nginx.conf
-rw-r--r-- 1 root root  636 May 28 13:22 scgi_params
-rw-r--r-- 1 root root  664 May 28 13:22 uwsgi_params


The most important file here is `nginx.conf` - it's the central nginx configuration file. Let's replace the default `nginx.conf` with the new configuration specified in the following cell.

In [5]:
%%writefile nginx_files/nginx.conf
events {

}

http {

    server {
        listen 80;
        return 200 "this is new message from nginx";
    }

}

Overwriting nginx_files/nginx.conf


Now replace the file with a new one and use `nginx -s reload` to make nginx use the new configuration.

In [6]:
%%bash
docker cp $(pwd)/nginx_files/nginx.conf experiment_nginx:/etc/nginx/nginx.conf
docker exec experiment_nginx nginx -s reload

2024/07/10 10:19:02 [notice] 67#67: signal process started


Here is showen how default behaviour of the nginx changed.

In [7]:
requests.get("http://localhost:80").content.decode("utf-8")

'this is new message from nginx'

We just got message specified in the `nginx.conf`.

## Reverse proxy

Another way to use nginx is as a reverse proxy, so you can redirect requests. You can define it with the `proxy_pass` directive in the `server` context.

The following config file makes requests to nginx to redirect to google.com.

In [3]:
%%writefile nginx_files/proxy_example.conf
events {

}

http {
    server {
        listen 80;
        location / {proxy_pass "https://google.com/";}
    }
}

Overwriting nginx_files/proxy_example.conf


Updating configuration and reloading nginx.

In [4]:
%%bash
docker cp $(pwd)/nginx_files/proxy_example.conf experiment_nginx:/etc/nginx/nginx.conf
docker exec experiment_nginx nginx -s reload

Now, by accessing `localhost:80`, we've got Google's main page back.

In [6]:
html_content = (
    requests
    .get("http://localhost:80")
    .content.decode("utf-8")
)
HTML(html_content)

,,Пашыраны пошук


## Logs

Logging in nginx is really usefull in process of the trouble shouting. There are two log files in nginx:

- `/var/log/nginx/access.log` - where all attempts to accept are logged.
- `/var/log/nginx/error.log` - where logged cases leading to errors.

For reasons that are still unclear, we need to reorder the log files in order to be able to read them. So in the following cell we do this for our container.

In [8]:
%%bash
docker exec -i experiment_nginx bash

rm /var/log/nginx/access.log /var/log/nginx/error.log
touch /var/log/nginx/access.log /var/log/nginx/error.log
nginx -s reload

And here is a request to nginx to print the contents of `access.log`.

In [9]:
!curl localhost:80 &> /dev/null
!docker exec experiment_nginx cat /var/log/nginx/access.log

172.17.0.1 - - [10/Jul/2024:10:19:08 +0000] "GET / HTTP/1.1" 200 30 "-" "curl/7.81.0"
